In [ ]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import json
import os
import pandas as pd

In [24]:
API_KEY = "xSxe6K-J8Pb8Lmc1X0zmBYIj-_0txNCroDSRgo4QxW2G5HOb2Qv0emTX-e4Bcsczs-Bi83ZwHNNenXIYzmI-V2bioFqc2vpXQf4QoV2DYrmP5iUxMkXGy4IJ7U0oZXYx"
CUISINE = "mediterranean"

In [25]:
# Using the yelp business search API: https://www.yelp.com/developers/documentation/v3/business_search
offset = 0
headers = {'Authorization': 'Bearer {}'.format(API_KEY)}
search_api_url = 'https://api.yelp.com/v3/businesses/search'
params = {
        'location': 'Manhattan, New York',
        'term': 'restaurants',
        'categories': CUISINE,
        'limit': 50,
}

In [26]:
# Fields to care about
# restaurantID, cuisine, name, address, coordinates, number_of_reviews, rating, zip_code
scraped_restaurants = 0
print(f"Collecting {CUISINE} restaurants")
with open(f"yelp_data_{CUISINE}.jsonl", 'a') as file_handle:
    while offset < 1000: # Yelp API only allows 1000 results
        print("Current Offset: ", offset)
        params["offset"] = offset
        response = requests.get(search_api_url, headers=headers, params=params)
        if response.status_code == 200:
            restaurants_information_json = response.json()
            print(f"Retreived {len(restaurants_information_json['businesses'])} results")
            if (len(restaurants_information_json['businesses'])) < 0:
                print("Stopping scraping as no more results are available")
                break
            # Iterate over all restaurants
            for restaurant in restaurants_information_json['businesses']:
                restaurant = {
                    "restaurantID": restaurant['id'],
                    "cuisine": CUISINE,
                    "name": restaurant['name'],
                    "address": restaurant['location']['address1'],
                    "coordinates": restaurant['coordinates'],
                    "number_of_reviews": restaurant['review_count'],
                    "rating": restaurant['rating'],
                    "zip_code": restaurant['location']['zip_code'],
                }
                file_handle.write(json.dumps(restaurant) + '\n')
            scraped_restaurants += len(restaurants_information_json['businesses'])
            print("Number of restaurants scraped: ", scraped_restaurants)
            offset += len(restaurants_information_json['businesses'])
        else:
            print("Error with the request")
            print(response.json())


Current Offset:  0
Retreived 50 results
Number of restaurants scraped:  50
Current Offset:  50
Retreived 50 results
Number of restaurants scraped:  100
Current Offset:  100
Retreived 50 results
Number of restaurants scraped:  150
Current Offset:  150
Retreived 50 results
Number of restaurants scraped:  200
Current Offset:  200
Retreived 50 results
Number of restaurants scraped:  250
Current Offset:  250
Retreived 50 results
Number of restaurants scraped:  300
Current Offset:  300
Retreived 50 results
Number of restaurants scraped:  350
Current Offset:  350
Retreived 50 results
Number of restaurants scraped:  400
Current Offset:  400
Retreived 50 results
Number of restaurants scraped:  450
Current Offset:  450
Retreived 50 results
Number of restaurants scraped:  500
Current Offset:  500
Retreived 37 results
Number of restaurants scraped:  537
Current Offset:  537
Retreived 0 results
Number of restaurants scraped:  537
Current Offset:  537
Retreived 0 results
Number of restaurants scrape

KeyboardInterrupt: 

In [74]:
restaurants_information_json

{'businesses': [{'id': 'CrteHKCnfqkFJWBWgEnPqA',
   'alias': 'tacos-los-2-carnales-new-york',
   'name': 'Tacos Los 2 Carnales',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/8sHj0A8KMauv0beIYFnv3Q/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/tacos-los-2-carnales-new-york?adjust_creative=KPUHHwsoXiJ0VA_ZMQ5kxA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KPUHHwsoXiJ0VA_ZMQ5kxA',
   'review_count': 2,
   'categories': [{'alias': 'foodtrucks', 'title': 'Food Trucks'},
    {'alias': 'tacos', 'title': 'Tacos'}],
   'rating': 5.0,
   'coordinates': {'latitude': 40.745145632560586,
    'longitude': -73.97829927612284},
   'transactions': [],
   'location': {'address1': '',
    'address2': None,
    'address3': '',
    'city': 'New York',
    'zip_code': '10016',
    'country': 'US',
    'state': 'NY',
    'display_address': ['New York, NY 10016']},
   'phone': '+19295076196',
   'display_phone': '(929) 507-6196',
   'distance': 1675.7

In [27]:
# Convert data to UTF-8 in post processing
cuisines = ["chinese", "french", "italian", "japanese", "mexican", "greek", "thai", "europian", "korean", "mediterranean"]
for cuisine in cuisines:
    src_handle = open(f"yelp_data_{cuisine}.jsonl", 'r')
    with open(f"yelp_data_{cuisine}_utf8.jsonl", 'w', encoding="utf-8") as destination_handle:
        for line in src_handle:
            loaded = json.loads(line)
            destination_handle.write(json.dumps(loaded, ensure_ascii=False) + '\n')
    src_handle.close()

In [28]:
# Turn the data into a merged dataframe and determine if we have duplicate restaurants
cuisines = ["chinese", "italian", "french", "japanese", "mexican", "greek", "thai", "europian", "korean", "mediterranean",]

master_dataframe = pd.DataFrame(columns=["restaurantID", "cuisine", "name", "address", "coordinates", "number_of_reviews", "rating", "zip_code"])
for cuisine in cuisines:
    master_dataframe = pd.concat([master_dataframe, pd.read_json(f"yelp_data_{cuisine}_utf8.jsonl", lines=True)], axis=0, join="outer", ignore_index=True)
len(master_dataframe)


6744

In [29]:
master_dataframe

,restaurantID,cuisine,name,address,coordinates,number_of_reviews,rating,zip_code
0,XsXLVWr1UZWVhKThNvNiaA,chinese,Blue Willow 夜来湘,40 W 56th St,"{'latitude': 40.76292, 'longitude': -73.976546}",969,4.5,10019
1,pBNMZp_tKCAfRNx7-ybcHQ,chinese,Calle Dao Chelsea,461 W 23rd St,"{'latitude': 40.74765439999999, 'longitude': -...",1047,4.5,10011
2,BNaveJmi-OUGKG9sRfrIhA,chinese,Mr. Sun,48 W 48th St,"{'latitude': 40.75809, 'longitude': -73.98026}",250,4.5,10036
3,UK1oGbr8GU2rufpnJR4-HQ,chinese,Joe's Home of Soup Dumplings,7 E 48th St,"{'latitude': 40.75723, 'longitude': -73.97702}",494,4.5,10017
4,oe8GEFE4QLFAKt87y7zcgA,chinese,Very Fresh Noodles,409 W 15th St,"{'latitude': 40.74207, 'longitude': -74.00565}",1459,4.5,10011
...,...,...,...,...,...,...,...,...
6739,gxVzT2FC6uk_0WmhLFs_MQ,mediterranean,Layaly Zamman Cafe,3196 Kennedy Blvd,"{'latitude': 40.7419093698263, 'longitude': -7...",0,0.0,7306
6740,SsnlMmxi7vgWa3k3_pyFRA,mediterranean,Aura,25-70 Steinway St,"{'latitude': 40.76638, 'longitude': -73.91328}",0,0.0,11103
6741,eLMeXJkpmZ_Rjp4FEs3oaA,mediterranean,Vegan Grill,187 Montrose Ave,"{'latitude': 40.70761, 'longitude': -73.9417}",0,0.0,11206
6742,cUAHyiaNYJ6vsFOna8g8_Q,mediterranean,Falafel,50 Greenwich St,"{'latitude': 40.7068642793082, 'longitude': -7...",2,3.0,10014


In [35]:
# Getting rid of duplicate restaurants and generate jsonl file for DynamoDB
with open('yelp_data2_utf8.jsonl', 'w', encoding='utf-8') as file:
    master_dataframe[~master_dataframe.duplicated(subset="restaurantID", keep=False) & ~master_dataframe["address"].isnull()].to_json("yelp_data2_utf8.jsonl", orient="records", lines=False, force_ascii=False)

In [38]:
# Getting rid of duplicate restaurants and generating json file for OpenSearch
_id = 1
index = "restaurants"
with open('yelp_utf8_opensearch.jsonl', 'w', encoding='utf-8') as file:
    for _, row in master_dataframe[~master_dataframe.duplicated(subset="restaurantID", keep=False) & ~master_dataframe["address"].isnull()].iterrows():
        file.write(json.dumps({"index": {"_index": index, "_id": _id}}, ensure_ascii=False) + '\n' + json.dumps({"restaurantID":str(row["restaurantID"]), "cuisine":str(row["cuisine"])}, ensure_ascii=False) + '\n')
        _id += 1